# What makes resturants close

In this part of the analysis we wish to investigate what factors makes resturants close.

# Load data

In [138]:
import pandas as pd
import numpy as np

In [146]:
Resturants=pd.read_csv('data/toronto_restaurant_business.csv',encoding="utf8")

In [30]:
len(Resturants)

8681

## Data preparation

First we remove data columns which decribe information which is only specific for the individual resturant. This will be the business id, the name of the resturant, address and since we are only intrested in resturants in Toronto also city and state. While longitude and latitude provide more precises information the information is overlapping with the postal_code, however the postal code provide a more general indication of the area and are therefore choosen to represent location.

In [147]:
Resturants

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,rVBPQdeayMYht4Uv_FOLHg,Gourmet Burger Company,843 Kipling Avenue,Toronto,ON,M8Z 5G9,43.633291,-79.531768,3.0,13,0,"{""RestaurantsPriceRange2"": ""2"", ""RestaurantsAt...","Restaurants, Burgers, Food",{}
1,0QjROMVW9ACKjhSEfHqNCQ,Mi Mi Restaurant,688 Gerrard Street E,Toronto,ON,M4M 1Y3,43.666376,-79.348773,4.0,116,1,"{""RestaurantsTakeOut"": ""True"", ""Alcohol"": ""bee...","Vietnamese, Restaurants","{""Monday"": ""11:00-22:00"", ""Tuesday"": ""11:00-22..."
2,8k62wYhDVq1-652YbJi5eg,Tim Hortons,90 Adelaide Street W,Toronto,ON,M5H 3V9,43.649859,-79.382060,3.0,8,1,"{""OutdoorSeating"": ""False"", ""RestaurantsDelive...","Bagels, Donuts, Food, Cafes, Coffee & Tea, Res...",{}
3,0DnQh8SE8BSnvJltGCCiWg,Chick-N-Joy,3-1265 York Mills Road,Toronto,ON,M3A 1Z3,43.765279,-79.326248,3.0,11,1,"{""NoiseLevel"": ""loud"", ""BusinessParking"": {""ga...","Fast Food, Restaurants, Chicken Shop",{}
4,NLaK58WvlNQdUunSIkt-jA,Zav Coffee Shop & Gallery,2048 Danforth Avenue,Toronto,ON,M4C 1J6,43.685608,-79.313936,4.5,24,1,"{""DogsAllowed"": ""False"", ""OutdoorSeating"": ""Tr...","Coffee & Tea, Restaurants, Sandwiches, Food","{""Monday"": ""0:00-0:00"", ""Tuesday"": ""7:30-17:00..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8676,thzyiQZb16zD8wMliaEfRQ,Sushi Supreme,1995 Yonge Street,Toronto,ON,M4S 1Z8,43.700617,-79.396762,4.0,104,1,"{""Ambience"": {""romantic"": ""False"", ""intimate"":...","Sushi Bars, Restaurants, Japanese","{""Monday"": ""12:00-23:00"", ""Tuesday"": ""12:00-23..."
8677,eUi3O-8Gqh_nJ_ZhO-25gQ,Get & Go Burrito,"1077 Wilson Avenue, Unit 8",Toronto,ON,M3K 1G7,43.726656,-79.480365,3.5,43,1,"{""BusinessParking"": {""garage"": ""False"", ""stree...","Mexican, Restaurants","{""Monday"": ""11:00-23:00"", ""Tuesday"": ""11:00-23..."
8678,yFQCdWr_k1pTObzHPGis9Q,Grasshopper Restaurant,310 College Street,Toronto,ON,M5T 1S2,43.657716,-79.402098,4.0,177,1,"{""DogsAllowed"": ""False"", ""BikeParking"": ""True""...","Vegan, Restaurants, Vegetarian, Salad","{""Monday"": ""11:30-22:00"", ""Tuesday"": ""11:30-22..."
8679,GAgEoHcf4PSuZRS5Zd3ltA,Q's Shawarma,1075 Martin Grove Road,Toronto,ON,M9W 4W6,43.701807,-79.575135,4.0,16,1,"{""GoodForKids"": ""True"", ""HasTV"": ""True"", ""Rest...","Restaurants, Mediterranean","{""Monday"": ""11:00-21:00"", ""Tuesday"": ""11:00-21..."


In [141]:
Resturants=Resturants.drop(['name','address','city','state','longitude','latitude','business_id'],axis=1)

To be able to work with the data postal_code is one-hot-encoded.

In [148]:
Resturants=pd.concat([Resturants,pd.get_dummies(Resturants.postal_code,prefix='postal_code',drop_first=False)],axis=1)
Resturants=Resturants.drop(['postal_code'],axis=1)

The atributtes ne

In [149]:
# attribute is transformed from string to dictionaries
Resturants.attributes=Resturants.attributes.apply(lambda d: eval(d))

As not all attributes are listed for all resturants we first need to get a list of all available attributes

In [154]:
attributes=set([])
for d in Resturants.attributes:
    attributes=attributes.union(set(d.keys()))
print("The are the following {} attributes:".format(len(attributes)))
attributes=list(attributes)
for attribute in attributes:
    print("- {}".format(attribute))

The are the following 37 attributes:
- Corkage
- GoodForMeal
- RestaurantsReservations
- RestaurantsCounterService
- HairSpecializesIn
- Ambience
- HappyHour
- WiFi
- DietaryRestrictions
- GoodForDancing
- RestaurantsPriceRange2
- CoatCheck
- AcceptsInsurance
- Caters
- RestaurantsDelivery
- RestaurantsAttire
- RestaurantsGoodForGroups
- BikeParking
- BYOB
- GoodForKids
- BestNights
- BusinessParking
- ByAppointmentOnly
- WheelchairAccessible
- DriveThru
- DogsAllowed
- BusinessAcceptsCreditCards
- BusinessAcceptsBitcoin
- HasTV
- RestaurantsTableService
- Smoking
- AgesAllowed
- Alcohol
- NoiseLevel
- Music
- RestaurantsTakeOut
- OutdoorSeating


Each attribute is now transformed into seperate columns, and then one hot encoded.

In [156]:
for attribute in attributes:
    Resturants[attribute]=Resturants.attributes.apply(lambda a:  a[attribute] if (attribute in a) else np.nan)
    print("{}: {}".format(attribute,Resturants[attribute].isna().sum()/len(Resturants[attribute].isna())))
#Maybe we should remove some of the attributes which there is almost nothin of. What should we do with the attribudes we dont have a value for... We can asume false,
#but it will have to be individualized for everything...

Corkage: 0.9935491302845294
GoodForMeal: 0.47909227047575165
RestaurantsReservations: 0.1968667204239143
RestaurantsCounterService: 0.999884805897938
HairSpecializesIn: 0.9997696117958761
Ambience: 0.21518258265176823
HappyHour: 0.8767423107936874
WiFi: 0.3319894021426103
DietaryRestrictions: 0.9986176707752563
GoodForDancing: 0.9474714894597397
RestaurantsPriceRange2: 0.1932957032599931
CoatCheck: 0.9563414353185117
AcceptsInsurance: 0.9997696117958761
Caters: 0.4589333026149061
RestaurantsDelivery: 0.2220942287754867
RestaurantsAttire: 0.26414007602810735
RestaurantsGoodForGroups: 0.2094228775486695
BikeParking: 0.3900472295818454
BYOB: 0.9906692777329801
GoodForKids: 0.22474369312291212
BestNights: 0.9486234304803594
BusinessParking: 0.1878815804630803
ByAppointmentOnly: 0.9672848750143993
WheelchairAccessible: 0.8267480704987905
DriveThru: 0.9649809929731598
DogsAllowed: 0.8729409054256422
BusinessAcceptsCreditCards: 0.968667204239143
BusinessAcceptsBitcoin: 0.9997696117958761
HasT

In [132]:
Resturants.OutdoorSeating.unique()
# Resturants=pd.concat([Resturants,pd.get_dummies(Resturants.postal_code,prefix='postal_code',drop_first=False)],axis=1)
#Resturants=Resturants.drop(['postal_code'],axis=1)

array(['False', 'True', nan, 'None'], dtype=object)

In [67]:
eval(Resturants.attributes[0]).keys()

dict_keys(['RestaurantsPriceRange2', 'RestaurantsAttire', 'Alcohol', 'Caters', 'HasTV', 'GoodForKids', 'RestaurantsGoodForGroups', 'RestaurantsTakeOut', 'RestaurantsDelivery', 'RestaurantsReservations', 'BusinessParking', 'NoiseLevel', 'WiFi', 'Ambience', 'OutdoorSeating'])

In [69]:
eval(Resturants.attributes[1])

{'RestaurantsTakeOut': 'True',
 'Alcohol': 'beer_and_wine',
 'GoodForKids': 'True',
 'RestaurantsPriceRange2': '1',
 'RestaurantsAttire': 'casual',
 'RestaurantsDelivery': 'False',
 'Ambience': {'romantic': 'False',
  'intimate': 'False',
  'classy': 'False',
  'hipster': 'False',
  'divey': 'False',
  'touristy': 'False',
  'trendy': 'False',
  'upscale': 'False',
  'casual': 'True'},
 'BusinessParking': {'garage': 'False',
  'street': 'True',
  'validated': 'False',
  'lot': 'False',
  'valet': 'False'},
 'RestaurantsGoodForGroups': 'True',
 'RestaurantsReservations': 'False',
 'OutdoorSeating': 'False',
 'NoiseLevel': 'average',
 'GoodForMeal': {'dessert': 'False',
  'latenight': 'False',
  'lunch': 'True',
  'dinner': 'True',
  'brunch': 'False',
  'breakfast': 'False'},
 'WiFi': 'no',
 'BikeParking': 'True',
 'RestaurantsTableService': 'True',
 'HasTV': 'True',
 'Caters': 'False'}

## Predicting if a business close

### Baseline

Random classification as a baseline

To predict which resturants close, we build a binary classifyer classifying wheter or not a business is closed or open. In other words we are intrested in building a binary classify. To do so logistic regression is used.

### Logistic regression

In [19]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression

In [ ]:
clf = LogisticRegression

## Interpreting factors influencing prediction

In [20]:
 from sklearn.datasets import load_iris